In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import os


In [ ]:
# Hàm để trích xuất đặc trưng HOG từ ảnh grayscale
def extract_hog_features(image):
    # Áp dụng HOG và trả về các đặc trưng
    features = hog(image, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
    return features


In [ ]:
# Đường dẫn tới thư mục chứa dữ liệu bàn tay
hand_dataset_path = 'signdata/Train'

# Hàm để load dữ liệu từ thư mục và trích xuất HOG features
# Thư mục chứa các folder từ A đến Z, mỗi folder chứa ảnh bàn tay, không có ảnh không có bàn tay
def load_hand_dataset(dataset_path):
    X, y = [], []
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for file_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, file_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                features = extract_hog_features(image)
                X.append(features)
                y.append(1)
    return np.array(X), np.array(y)

    X = []
    y = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):
                img_path = os.path.join(root, file)
                # Đọc ảnh grayscale
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                # Trích xuất đặc trưng HOG
                features = extract_hog_features(img)
                X.append(features)
                # Xác định nhãn (1: là bàn tay, 0: không phải bàn tay)
                label = 1 if 'hand' in root else 0
                y.append(label)
    return np.array(X), np.array(y)

# Load dữ liệu từ thư mục
X, y = load_hand_dataset(hand_dataset_path)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình SVM
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = svm_model.predict(X_test)

# Đánh giá độ chính xác của mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Độ chính xác của mô hình là: {accuracy * 100:.2f}%')


In [ ]:
def predict_hand(image_path, model):
    # Đọc ảnh và chuyển đổi thành grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Trích xuất đặc trưng HOG từ ảnh
    features = extract_hog_features(img)
    # Reshape features vì mô hình cần đầu vào là 2D array
    features = features.reshape(1, -1)
    # Dự đoán
    prediction = model.predict(features)
    if prediction[0] == 1:
        return "This is likely a hand."
    else:
        return "This is not a hand."

# Dự đoán trên ảnh mới
image_path = 'path_to_your_test_image.jpg'
prediction_result = predict_hand(image_path, svm_model)
print(prediction_result)
